In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_excel("Concrete_Data.xls")
data = pd.concat([pd.Series(1, index = data.index, name = "bias"), data], axis = 1)
data

,bias,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,1,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,1,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,1,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,1,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075
...,...,...,...,...,...,...,...,...,...,...
1025,1,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.284354
1026,1,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.178794
1027,1,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.696601
1028,1,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.768036


In [3]:
data_array = np.array(data)

x = data_array[:,:-1]
y = data_array[:,-1]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=130, random_state=13)
x_train_t = np.transpose(x_train)
x_test_t = np.transpose(x_test)

B) Multi-variate linear regression 
y = f(x) = (m · x)     (i.e., dot product between m and x),  
where m = (m0, m1, ..., mp)T and x = (1, x1, ..., xp)T, with m0 in place of b in the model.  

In [4]:
def mse_function(x, y, m):
    items_number, features_number = x.shape
    y_predict = np.dot(np.transpose(m), np.transpose(x))
    mse = (1/items_number) * np.sum(np.square(y - y_predict))
    return mse

In [5]:
def derivative_m(x, y, m):
    items_number , features_number = x.shape
    y_predict = np.dot(np.transpose(m), np.transpose(x))
    grad = (-2/items_number) * (np.dot((y - y_predict), x))
    return np.reshape(grad, (features_number,1))

In [6]:
def multivariate_linear_regression(x, y, max_iteration, learning_rate):
    features_number = x.shape[1]
    #initial the m matrix(900, 1)
    m = np.random.uniform(0, 1, (features_number, 1))
    iteration = 0
    mse = mse_function(x, y, m)
    pre_mse = float("inf")
    
    while iteration < max_iteration:
    # while mse - pre_mse > 0.00001:
        pre_mse = mse
        m = m - learning_rate * derivative_m(x, y, m)
        mse = mse_function(x, y, m)
        iteration += 1
        if mse > pre_mse:
            learning_rate *= 0.8
        else:
            learning_rate *= 1.1
    
    variance =  1 - (mse / (np.max(y) - np.min(y)))
    predict_y = np.dot(np.transpose(m), np.transpose(x))
    return m, mse, predict_y, variance

In [7]:
max_iteration = 1000000
learning_rate = 0.000000001
m, train_mse, predict_y, variance = multivariate_linear_regression(x_train, y_train, max_iteration, learning_rate)
test_mse = mse_function(x_test, y_test, m)
print("The m values: \n", m)
print("Train MSE: ", train_mse)
print("Test MSE: ", test_mse)
print("Explained variance for train dataset: ", variance)
print("Explained variance for test dataset: ", 1 - (test_mse)/(np.max(y_test)-np.min(y_test)))

The m values: 
 [[ 0.11053072]
 [ 0.11258366]
 [ 0.09666151]
 [ 0.07548495]
 [-0.18196253]
 [ 0.27267005]
 [ 0.01117704]
 [ 0.01052633]
 [ 0.11085249]]
Train MSE:  105.22458452893108
Test MSE:  122.38094836451782
Explained variance for train dataset:  -0.31092526087989936
Explained variance for test dataset:  -0.5855299219828167


C) Optional extension 1 – Mean Absolute Error as the loss function 
y = f(x) = (m · x)     (i.e., dot product between m and x),  
where m = (m0, m1, ..., mp)T and x = (1, x1, ..., xp)T, with m0 in place of b in the model.  

In [8]:
def mae_function(x, y, m):
    items_number, features_number = x.shape
    y_predict = np.dot(np.transpose(m), np.transpose(x))
    mae = (1/items_number) * np.sum(np.abs(y - y_predict))
    return mae

In [9]:
def multivariate_linear_regression_mae(x, y, max_iteration, learning_rate):
    features_number = x.shape[1]
    #initial the m matrix(900, 1)
    m = np.random.uniform(0, 1, (features_number, 1))
    iteration = 0
    mae = mae_function(x, y, m)
    pre_mae = float("inf")
    
    while iteration < max_iteration:
        pre_mae = mae
        m = m - learning_rate * derivative_m(x, y, m)
        mae = mae_function(x, y, m)
        if mae > pre_mae:
            learning_rate *= 0.8
        else:
            learning_rate *= 1.1
        iteration += 1
    
    predict_y = np.dot(np.transpose(m), np.transpose(x))
    variance =  1 - (mae / (np.max(y) - np.min(y)))
    return m, mae, predict_y, variance

In [10]:
max_iteration = 1000000
learning_rate = 0.00000001
m, train_mae, predict_y, variance = multivariate_linear_regression_mae(x_train, y_train, max_iteration, learning_rate)
test_mae = mae_function(x_test, y_test, m)
print("The m values: \n", m)
print("Train MAE: ", train_mae)
print("Test MAE: ", test_mae)
print("Explained variance for train dataset: ", variance)
print("Explained variance for test dataset: ", 1 - (test_mae)/(np.max(y_test)-np.min(y_test)))

The m values: 
 [[ 0.52754337]
 [ 0.11246958]
 [ 0.09652626]
 [ 0.07533291]
 [-0.18254003]
 [ 0.27215925]
 [ 0.01103613]
 [ 0.01036944]
 [ 0.11084857]]
Train MAE:  8.199316146645993
Test MAE:  8.649318487614611
Explained variance for train dataset:  0.8978500061872579
Explained variance for test dataset:  0.8879420902506407


D) Optional extension 2 – Data pre-processing (Normalization)
y = f(x) = (m · x)     (i.e., dot product between m and x),  
where m = (m0, m1, ..., mp)T and x = (1, x1, ..., xp)T, with m0 in place of b in the model.  

 Histograms of feature values after normalizing
 Normalization Formula: v_normalized = (v - min)/(max - min)

In [11]:
from sklearn import preprocessing
def normalization(data):
    result = data.copy()
    for feature_name in data.columns:
        max_value = data[feature_name].max()
        min_value = data[feature_name].min()
        result[feature_name] = (data[feature_name] - min_value) / (max_value - min_value)
    return result

In [12]:
data = pd.read_excel("Concrete_Data.xls")
normalized_data = normalization(data)
features = normalized_data.columns
normalized_data = pd.concat([pd.Series(1, index = normalized_data.index, name = "bias"), normalized_data], axis = 1)
normalized_data_array = np.array(normalized_data)
normalized_data.shape 

(1030, 10)

In [13]:
normalized_x = normalized_data_array[:,:-1]
normalized_y = normalized_data_array[:,-1]
normalized_x_train, normalized_x_test, normalized_y_train, normalized_y_test = train_test_split(normalized_x, normalized_y, test_size=130, random_state=13)
normalized_x_train_t = np.transpose(x_train)
normalized_x_test_t = np.transpose(x_test)

In [14]:
max_iteration = 1000000
learning_rate = 0.00000001
m, train_mse, predict_y, variance = multivariate_linear_regression(normalized_x_train, normalized_y_train, max_iteration, learning_rate)
test_mse = mse_function(normalized_x_test, normalized_y_test, m)
print("The m values: \n", m)
print("After Normalization, Train MSE: ", train_mse)
print("After Normalization, Test MSE: ", test_mse)
print("After Normalization, Explained variance for train dataset: ", variance)
print("After Normalization, Explained variance for test dataset: ", 1 - (test_mse)/(np.max(y_test)-np.min(y_test)))


The m values: 
 [[-0.22531384]
 [ 0.71329528]
 [ 0.52907059]
 [ 0.24842999]
 [-0.14069416]
 [ 0.14195362]
 [ 0.14388145]
 [ 0.1761126 ]
 [ 0.5055289 ]]
After Normalization, Train MSE:  0.016228432193953143
After Normalization, Test MSE:  0.019959090100068227
After Normalization, Explained variance for train dataset:  0.9837715678060469
After Normalization, Explained variance for test dataset:  0.9997414161681853
